In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.8/486.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 20.3 MB/s eta 0:00:00
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.5.4.60
    Uninstalling opencv-python-4.5.4.60:
      Successfully uninstalled opencv-python-4.5.4.60


In [2]:
import os
import shutil
import random
import json
import glob
import torch

from ultralytics import YOLO
from PIL import Image

In [3]:
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.13.0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


In [4]:
input_folder = "/kaggle/input/ui-kits-simple/rico_dataset_v0.1_semantic_annotations/semantic_annotations"
output_image_folder = "/kaggle/working/data/rico/images"
output_json_folder = "/kaggle/working/data/rico/jsons"

os.makedirs(output_image_folder, exist_ok=True)
os.makedirs(output_json_folder, exist_ok=True)

for file_name in os.listdir(input_folder):
    if file_name.endswith(".png") or file_name.endswith(".jpg"):
        shutil.copy(os.path.join(input_folder, file_name), os.path.join(output_image_folder, file_name))
    elif file_name.endswith(".json"):
        shutil.copy(os.path.join(input_folder, file_name), os.path.join(output_json_folder, file_name))


In [5]:
#bounds => [left, top, right, bottom]

input_path = "/kaggle/working/data/rico/jsons"
output_path = '/kaggle/working/data/rico/txt'

for filename in os.listdir(input_path):
    with open(os.path.join(input_path, filename)) as f:
        data = json.load(f)

        def get_child_components(data, labels=[]):
            if 'children' in list(data.keys()):
                for c in data['children']:

                    label_map = {
                        'Image': 0,
                        'Text': 1,
                        'Text Button': 2,
                        'Input': 3,
                        'Icon': 4,
                        'List Item': 5,
                        'Another': 6,
                        # more mappings if needed
                    }

                    #Convert these bounds to YOLO format

                    width = c['bounds'][2] - c['bounds'][0]
                    height = c['bounds'][3] - c['bounds'][1]

                    x_center = c['bounds'][0] + width/2
                    y_center = c['bounds'][1] + height/2
                    
                    base = data['bounds']
                    
                    image_width = 1440 #int(base[2])
                    image_height = 2560 #int(base[3])
                    
                    x_center_norm = x_center/image_width
                    y_center_norm = y_center/image_height
                    width_norm = width/image_width
                    height_norm = height/image_height

                    component_index = label_map.get(c['componentLabel'], 0)

                    yolo_format = [component_index, x_center_norm, y_center_norm, width_norm, height_norm]

                    labels.append({c['componentLabel']: yolo_format})

                    get_child_components(c, labels)
            return labels

        try:
            os.mkdir(output_path)
        except:
            pass
        
        output_file_path = os.path.join(output_path, filename[:-5] + '.txt')
        
        with open(output_file_path, 'w') as f:
            for i in get_child_components(data):
                for k in i.keys():
                    f.write('{} {} {} {} {}\n'.format(int(i[k][0]), float(i[k][1]), float(i[k][2]), float(i[k][3]), float(i[k][4])))


In [6]:
"""Finds corresponding png files after distributed txt files"""

# Define the paths to the image and text files
image_path = "/kaggle/working/data/rico/images"
text_path = "/kaggle/working/data/rico/txt"

# Define the output directories for the train, val, and test sets
train_path = "/kaggle/working/data/rico/train"
val_path = "/kaggle/working/data/rico/val"
test_path = "/kaggle/working/data/rico/test"

# Define the ratio of the train and val sets
train_ratio = 0.70
val_ratio = 0.15
test_ratio = 0.15

# Get the list of text files
text_files = sorted(os.listdir(text_path))

# Shuffle the list of text files
random.shuffle(text_files)

# Calculate the number of files for each set
num_train = int(len(text_files) * train_ratio)
num_val = int(len(text_files) * val_ratio)
num_test = int(len(text_files) - num_train - num_val)

# Assign each file to a set
train_files = text_files[:num_train]
val_files = text_files[num_train:num_train+num_val]
test_files = text_files[num_train+num_val:]

# Create the output directories, if exists delete them and create new ones
if os.path.exists(train_path):
    shutil.rmtree(train_path)
os.makedirs(train_path)

if os.path.exists(val_path):
    shutil.rmtree(val_path)
os.makedirs(val_path)

if os.path.exists(test_path):
    shutil.rmtree(test_path)
os.makedirs(test_path)

# Copy train txt files to the train directory, then copy the corresponding png files
for file in train_files:
    shutil.copy(os.path.join(text_path, file), train_path)
    shutil.copy(os.path.join(image_path, file[:-4] + ".png"), train_path)

# Copy val txt files to the val directory, then copy the corresponding png files
for file in val_files:
    shutil.copy(os.path.join(text_path, file), val_path)
    shutil.copy(os.path.join(image_path, file[:-4] + ".png"), val_path)

# Copy test txt files to the test directory, then copy the corresponding png files
for file in test_files:
    shutil.copy(os.path.join(text_path, file), test_path)
    shutil.copy(os.path.join(image_path, file[:-4] + ".png"), test_path)

In [7]:
# Print the number of files in each set, both text and image files
print("Number of train files: ", len(os.listdir(train_path)))
print("Number of val files: ", len(os.listdir(val_path)))
print("Number of test files: ", len(os.listdir(test_path)))

Number of train files:  92764
Number of val files:  19878
Number of test files:  19880


In [8]:
# Load a model
# model_path = "/kaggle/input/core-components/model.yaml"

model = YOLO('yolov8n.yaml') # build a new model from YAML
model = YOLO('yolov8n.pt',task='detect')  # load a pretrained model (recommended for training)
# model = YOLO(yaml_path, task='detect').load('yolov8n.pt') # build from YAML and transfer weights


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.Conv                  [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.Conv                  [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.C2f                   [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.Conv                  [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.C2f                   [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.Conv                  [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.C2f                   [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.Conv                  [128

  0%|          | 0.00/6.23M [00:00<?, ?B/s]

In [9]:
# Train the model
yaml_path = "/kaggle/input/core-components/data.yaml"
device = 'cuda:0'  # Use the first available GPU
model.train(data=yaml_path, epochs=2, imgsz=640, device=device, val=True)

Ultralytics YOLOv8.0.58 🚀 Python-3.7.12 torch-1.13.0 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/input/core-components/data.yaml, epochs=2, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8

  0%|          | 0.00/755k [00:00<?, ?B/s]

Overriding model.yaml nc=80 with nc=7

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.Conv                  [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.Conv                  [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.C2f                   [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.Conv                  [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.C2f                   [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.Conv                  [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.C2f                   [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

In [10]:
# Export the model as torchscript
best_weight_path = '/kaggle/working/runs/detect/train/weights/best.pt'
model = YOLO(best_weight_path) 
model.fuse()  
model.info(verbose=True)  # Print model information
model.export()

Model summary (fused): 168 layers, 3007013 parameters, 0 gradients, 8.1 GFLOPs
Model summary (fused): 168 layers, 3007013 parameters, 0 gradients, 8.1 GFLOPs
Ultralytics YOLOv8.0.58 🚀 Python-3.7.12 torch-1.13.0 CPU

PyTorch: starting from /kaggle/working/runs/detect/train/weights/best.pt with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 11, 8400) (5.9 MB)

TorchScript: starting export with torch 1.13.0...
TorchScript: export success ✅ 1.9s, saved as /kaggle/working/runs/detect/train/weights/best.torchscript (11.9 MB)

Export complete (2.1s)
Results saved to /kaggle/working/runs/detect/train/weights
Predict:         yolo predict task=detect model=/kaggle/working/runs/detect/train/weights/best.torchscript imgsz=640 
Validate:        yolo val task=detect model=/kaggle/working/runs/detect/train/weights/best.torchscript imgsz=640 data=/kaggle/input/core-components/data.yaml 
Visualize:       https://netron.app


'/kaggle/working/runs/detect/train/weights/best.torchscript'

In [11]:
# Test model with a random screenshot
dir_path = "/kaggle/input/ui-kits-simple/unique_uis/combined"
jpg_files = glob.glob(dir_path + "/*.jpg")

if jpg_files:
    try:
        random_jpg = random.choice(jpg_files)
        image = Image.open(random_jpg)
        result = model.predict(source=image, save=True)
        print(result)
        
    except Exception as e:
        print("Error occurred while processing image:", e)
        
else:
    print("No JPG files found in directory.")


0: 640x384 7 Images, 50 Texts, 2 Text Buttons, 5 Icons, 4 List Items, 18.7ms
Speed: 0.4ms preprocess, 18.7ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict


[ultralytics.yolo.engine.results.Results object with attributes:

_keys: ('boxes', 'masks', 'probs')
boxes: ultralytics.yolo.engine.results.Boxes object
keys: ['boxes']
masks: None
names: {0: 'Image', 1: 'Text', 2: 'Text Button', 3: 'Input', 4: 'Icon', 5: 'List Item', 6: 'Another'}
orig_img: array([[[  7,  98, 237],
        [  7,  98, 237],
        [  7,  98, 237],
        ...,
        [  7,  98, 237],
        [  7,  98, 237],
        [  7,  98, 237]],

       [[  7,  98, 237],
        [  7,  98, 237],
        [  7,  98, 237],
        ...,
        [  7,  98, 237],
        [  7,  98, 237],
        [  7,  98, 237]],

       [[  7,  98, 237],
        [  7,  98, 237],
        [  7,  98, 237],
        ...,
        [  7,  98, 237],
        [  7,  98, 237],
        [  7,  98, 237]],

       ...,

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [

In [12]:
# import shutil, os

# def kaggle_export(output_dir, zip_name):
#     """Create a zip file and export your work in the kaggle working directory.

#     Args:
#         output_dir (str): The path to the directory to be zipped.
#         zip_name (str): The name of the zip file to be created.

#     Returns:
#         None
#     """
#     if not os.path.exists(output_dir):
#         print(f"[Error]: output directory '{output_dir}' does not exist")
    
#     else:
#         try:
#             shutil.make_archive(zip_name, 'zip', output_dir)
#             print(f"[Success]: Your zip file is successfully created!")

#         except Exception as e:
#             print(f"[Error]: An error occurred while creating zip: {e}")

# kaggle_export("/kaggle/working/runs/detect/train", "model_result")